### Step1. Check GPU Usage ###
If the memory usage below is not 1MiB, it means that a task is already running within the instance.

In [1]:
!nvidia-smi

Sun Jul 27 13:20:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10G                    On  |   00000000:00:1E.0 Off |                    0 |
|  0%   31C    P8             10W /  300W |       1MiB /  23028MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### ComfyUI Basic Environment Setup ###
This cell handles the installation of all necessary basic programs and environment configuration required to run ComfyUI. The script consists of four main parts:

Git and Git-LFS Installation
git: A version control tool needed to download ComfyUI source code from GitHub.

git-lfs: An essential extension program for downloading large files such as AI models that can be several gigabytes (GB) in size.

The code checks if these two programs are already installed, and only proceeds with installation if they're not present, saving time.

ComfyUI Installation
Uses the git clone command to download the latest ComfyUI source code from GitHub.

Downloaded files are saved in a folder named ComfyUI under the current directory.

Essential Python Library Installation
Installs various Python libraries (PyTorch, NumPy, etc.) necessary for ComfyUI to function properly.

The requirements.txt file acts as an 'installation guide' containing the list of necessary libraries, and is used to perform the installation all at once.

ComfyUI-Manager Installation
Installs ComfyUI-Manager, an essential extension for ComfyUI.

This manager allows users to easily search and install needed custom nodes or models within the ComfyUI interface with just a few clicks, making it very convenient.

In conclusion, running this cell provides users with a clean basic environment ready to run ComfyUI without going through complicated installation processes.

In [ ]:
# --- Cell 1: Basic Environment Setup and ComfyUI Installation (Final Modified Version) ---
import os
import sys
import subprocess

# Install git and git-lfs (if not present in SageMaker environment)
print(">>> Checking Git and Git-LFS installation...")
try:
    subprocess.run(["git", "--version"], check=True, capture_output=True)
    subprocess.run(["git-lfs", "--version"], check=True, capture_output=True)
    print(">>> Git and Git-LFS are already installed.")
except (subprocess.CalledProcessError, FileNotFoundError):
    print(">>> Git or Git-LFS is not installed. Starting installation...")
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'gitpython'], check=True)
    subprocess.run(['conda', 'install', '-c', 'conda-forge', 'git-lfs', '-y'], check=True)
    subprocess.run(['git-lfs', 'install'], check=True)
    print(">>> Git and Git-LFS installation completed.")

# ComfyUI Installation
COMFYUI_DIR = os.path.join(os.getcwd(), "ComfyUI")
if not os.path.exists(COMFYUI_DIR):
    print(f">>> Cloning ComfyUI to '{COMFYUI_DIR}'...")
    subprocess.run(["git", "clone", "https://github.com/comfyanonymous/ComfyUI.git"], check=True)
    print(">>> ComfyUI cloning completed.")
else:
    print(f">>> ComfyUI already exists in '{COMFYUI_DIR}'.")

# Install required Python libraries
print(">>> Installing essential Python libraries...")
%cd {COMFYUI_DIR}

# 1. First install ComfyUI's basic library list
subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], check=True)

### ✨ Special Measures to Prevent Dependency Conflicts ✨ ###
# To resolve compatibility issues with SageMaker environment and complex dependency problems,
# We forcibly install stable version combinations found through debugging.
# This part is crucial in preventing all problems in advance.
print("\n>>> [IMPORTANT] Fixing major library versions to prevent dependency conflicts...")
subprocess.run([sys.executable, '-m', 'pip', 'install', '--upgrade',
                'transformers==4.49.0',
                'tensorflow',
                'jax',
                'jaxlib',
                'ml_dtypes',
                'protobuf>=3.20.3'
               ], check=True)
print(">>> Version fixing completed.")
### ✨ --- End of Special Measures --- ✨ ###

# 2. Install additional libraries
subprocess.run([sys.executable, '-m', 'pip', 'install', 'websocket-client', 'requests', 'numpy', 'Pillow', 'opencv-python'], check=True)
print("\n>>> All library installations completed.")

# Install ComfyUI-Manager (facilitates model management)
MANAGER_DIR = os.path.join(COMFYUI_DIR, "custom_nodes", "ComfyUI-Manager")
if not os.path.exists(MANAGER_DIR):
    print(">>> Installing ComfyUI-Manager...")
    subprocess.run(["git", "clone", "https://github.com/ltdrdata/ComfyUI-Manager.git", MANAGER_DIR], check=True)
    manager_req_path = os.path.join(MANAGER_DIR, "requirements.txt")
    if os.path.exists(manager_req_path):
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', manager_req_path], check=True)
    print(">>> ComfyUI-Manager installation completed.")
else:
    print(">>> ComfyUI-Manager is already installed.")

%cd ..
print("\n--- Environment setup has been perfectly completed. ---")

>>> Checking Git and Git-LFS installation...
>>> Git or Git-LFS is not installed. Starting installation...
Retrieving notices: done
Channels:
 - conda-forge
Platform: linux-64

### AI Model Download and Preparation ###
This cell handles downloading various AI models that will be actually used in ComfyUI workflows for image generation and places them in designated locations.

Folder Structure Creation
ComfyUI needs models to be stored in specific folders for proper recognition. This script automatically creates the following subfolders within the ComfyUI/models/ directory to store AI models:

checkpoints: Core model folder that serves as the foundation for image generation

controlnet: Folder for models that control image composition and poses

ipadapter: Folder for models that copy styles from other images

loras: Folder for small models that add specific characters or art styles

upscale_models: Folder for models that increase image resolution

Download Automation (download_model function)
The script defines a download_model function that automates model downloading. This function includes the following smart features:

Duplication Prevention: Checks if the file already exists before downloading to prevent unnecessary re-downloads

Automatic Download: Uses wget command to download models from specified URLs only when files are not present

Major Model Downloads
The script downloads essential models needed for workflows using the above function:

① Checkpoint: The most basic and important model that acts as the painter(👨‍🎨). In this example, it downloads the Juggernaut XL model, which excels in realistic expressions.

② ControlNet: Assistant models that help create more precise results by controlling image composition, depth, and outlines according to user preferences.

③ IP-Adapter: Models that apply styles (colors, mood, texture, etc.) from other images to newly generated images. The CLIP Vision model acts as eyes(👁️) needed for IP-Adapter to "recognize" the style of reference images.

④ Upscale: Post-processing models that increase image resolution without quality degradation.

In conclusion, running this cell prepares all 'ingredients (models)' needed for image generation, allowing ComfyUI to immediately perform tasks.

In [ ]:
# --- Cell 2: Download Models Required for Workflow ---
import os
import subprocess

# Set directory paths for storing models
CHECKPOINT_DIR = os.path.join(COMFYUI_DIR, "models", "checkpoints")
CONTROLNET_DIR = os.path.join(COMFYUI_DIR, "models", "controlnet")
IPADAPTER_DIR = os.path.join(COMFYUI_DIR, "models", "ipadapter")
CLIP_VISION_DIR = os.path.join(COMFYUI_DIR, "models", "clip_vision")
LORA_DIR = os.path.join(COMFYUI_DIR, "models", "loras")
UPSCALE_DIR = os.path.join(COMFYUI_DIR, "models", "upscale_models")

# Create directories
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(CONTROLNET_DIR, exist_ok=True)
os.makedirs(IPADAPTER_DIR, exist_ok=True)
os.makedirs(CLIP_VISION_DIR, exist_ok=True)
os.makedirs(LORA_DIR, exist_ok=True)
os.makedirs(UPSCALE_DIR, exist_ok=True)

# Download function
def download_model(url, dest_dir):
    filename = url.split('/')[-1]
    dest_path = os.path.join(dest_dir, filename)
    if not os.path.exists(dest_path):
        print(f">>> Downloading '{filename}'...")
        subprocess.run(["wget", "-c", url, "-O", dest_path], check=True)
        print(f">>> '{filename}' download completed.")
    else:
        print(f">>> '{filename}' already exists.")

# 1. SDXL Checkpoint Model
download_model("https://huggingface.co/RunDiffusion/Juggernaut-XL/resolve/main/juggernautXL_version2.safetensors", CHECKPOINT_DIR)

# 2. ControlNet Models (Depth and Canny)
download_model("https://huggingface.co/xinsir/controlnet-union-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors", CONTROLNET_DIR)

# 3. IP-Adapter Model
download_model("https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors", IPADAPTER_DIR)

# CLIP Vision Encoder
download_model("https://huggingface.co/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/resolve/main/model.safetensors", CLIP_VISION_DIR)

# 4. Upscale Model (4x-UltraSharp)
download_model("https://huggingface.co/lokCX/4x-UltraSharp/resolve/main/4x-UltraSharp.pth", UPSCALE_DIR)

print("\n--- All model downloads have been completed. ---")